In [2]:
import db_builder.db_handler as dbh
import os
import numpy as np
import cv2
from torchvision import transforms
from tqdm import tqdm
import datetime
from os import path

In [6]:
db_params = {
    'user': 'postgres',
    'password': input('Please enter DB pw'),  # enter your DB password
    'host': 'localhost',  # 'localhost' or IP address
    'port': '5432',  # '5432'
    'database': 'test', #tensionTerminator
}

Please enter DB pw DiBSE21


In [7]:
toolcheck = dbh.DB_Conn(db_params)
toolcheck.connect()
filepath_with_trigger = toolcheck.get_filepath_with_trigger()
filepath_with_duoballs = toolcheck.get_filepath_with_duoballs()
toolcheck.disconnect()

Connected to PostgreSQL, DB: test
2023-11-12 15:35:48,022 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2023-11-12 15:35:48,023 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-11-12 15:35:48,026 INFO sqlalchemy.engine.Engine select current_schema()
2023-11-12 15:35:48,027 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-11-12 15:35:48,029 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2023-11-12 15:35:48,030 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-11-12 15:35:48,031 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-12 15:35:48,036 INFO sqlalchemy.engine.Engine SELECT loop.id AS loop_id, loop.timestamp AS loop_timestamp, loop.session_length AS loop_session_length, loop.location AS loop_location, loop.human_labeled AS loop_human_labeled, loop.user_id AS loop_user_id 
FROM loop JOIN tools_loop_association ON loop.id = tools_loop_association.loop_id 
WHERE loop.human_labeled = true AND tools_loop_association.tool_id = %(tool_id_1)s
2023-11-12 15:35:4

In [14]:
def trigger_crop(image):
    crop_box = (400, 450, 550, 550)
    cropped_image = transforms.functional.crop(image, *crop_box)
    return cropped_image

In [15]:
def video_to_image_converter(source_path: str, output_path: str, crop=False):
    vidcap = cv2.VideoCapture(source_path)
    os.makedirs(output_path, exist_ok=True)
    count = 0

    total_frames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = int(vidcap.get(cv2.CAP_PROP_FPS))
    fps = 1

    while count < total_frames:
        success, image = vidcap.read()

        if count % int(fps) == 0:
            if not image is None and not image.size == 0:
                transform_test = transforms.ToPILImage()
                image = transform_test(image)
                if crop:
                    image = trigger_crop(image)
                image = np.asarray(image)
                cv2.imwrite(f"{output_path}/{count}.png", image)  # save frame as PNG file

        count += 1

    vidcap.release()
    cv2.destroyAllWindows()

In [16]:
def get_time():
    return datetime.datetime.now().strftime("%d_%m_%Y_%H_%M_%S")

In [17]:
def save_path(label: str, time):
    abs_path = os.getcwd()
    two_up =  path.abspath(path.join(abs_path ,"../.."))
    return path.join(two_up, 'data', f'{os.path.basename(abs_path)}', f'{time}', label)

In [18]:
current_time = get_time()

In [ ]:
for x in tqdm(filepath_with_trigger):
    video_to_image_converter(x, save_path('trigger', current_time), False)

 12%|█▏        | 9/74 [22:44<2:06:10, 116.47s/it]

In [ ]:
for x in tqdm(filepath_with_duoballs):
    video_to_image_converter(x, save_path('duoballs', current_time), False)